In [ ]:
#!import "../0-setup/Packages.ipynb"

# Azure Subscription

This model is based on the subscription webhooks received from the Azure Marketplace.
The landing page creates these facts.
The portal subscribes to them to determine how many replicators can be configured.

## Enterprise Application Pattern

The Azure landing page app is an enterprise application.
The top-level user is a temporary principal.
It is created to initialize the environment and the first administrator.
And then the private key is discarded.

In [ ]:
[FactType("Azure.Environment")]
public record AzureEnvironment(User creator, string environmentId);

Renderer.RenderTypes(typeof(AzureEnvironment))

Administrators represent people.
Administrators are granted permission to add service principals and other administrators to an environment.
That permission can be revoked.

The creator of the environment grants permission to the initial administrator.
Thereafter, an administrator can add new administrators and revoke permissions.

In [ ]:
[FactType("Azure.Administrator")]
public record AzureAdministrator(User user, AzureEnvironment environment, DateTime createdAt);

[FactType("Azure.Administrator.Revoke")]
public record RevokeAzureAdministrator(AzureAdministrator administrator);

Renderer.RenderTypes(typeof(AzureAdministrator), typeof(RevokeAzureAdministrator))

## Azure Marketplace Service Principals

Service principals represents machines.
Different machines play different roles within the process.
The model represents each role as a different type, so that authorization and distribution rules can be granular.

Azure Marketplace service principals run the marketplace admin site and landing page.
They respond to user input, administrator input, and webhooks.
They can record facts about user identities and subscriptions.

Portal service principals run the Replicator portal.
They connect the user and their replicators to Azure user identities and subscriptions.
They use that information to enable and disable endpoints in the multitenant replicator and API gateway.

An administrator creates service principals.
Service principals cannot grant privileges to others.

In [ ]:
[FactType("Azure.ServicePrincipal")]
public record AzureServicePrincipal(User user, AzureEnvironment environment, DateTime createdAt);

[FactType("Azure.ServicePrincipal.Revoke")]
public record RevokeAzureServicePrincipal(AzureServicePrincipal servicePrincipal);

Renderer.RenderTypes(typeof(AzureServicePrincipal), typeof(RevokeAzureServicePrincipal))

## Subscriptions

Azure informs the landing page app of subscription activities by and related to a user.
It does so through webhook calls.
The landing page app also permits that user to take actions on the subscription through its own user interface.

In [ ]:
[FactType("Azure.Subscription")]
public record Subscription(AzureEnvironment environment, Guid subscriptionId);

[FactType("Azure.Subscription.Name")]
public record SubscriptionName(Subscription subscription, string value, SubscriptionName[] prior);

[FactType("Azure.Subscription.Unsubscribe")]
public record Unsubscribe(Subscription subscription);

Renderer.RenderTypes(typeof(Subscription), typeof(Unsubscribe))

This model also represents the user by their identifier.
The model does not treat Azure users as Jinaga users.
It records information _about_ the user, not _by_ the user.

A user can transfer a subscription to another user.
This is modeled using the Entity Reference pattern, a kind of mutable property.

In [ ]:
[FactType("Azure.UserIdentity")]
public record UserIdentity(AzureEnvironment environment, string userId);

[FactType("Azure.Subscription.UserIdentity")]
public record SubscriptionUserIdentity(Subscription subscription, UserIdentity userIdentity, SubscriptionUserIdentity[] prior);

Renderer.RenderTypes(typeof(SubscriptionUserIdentity))

## Subscription Activities

A user can adjust the plan to which a subscription applies.
This is captured as a mutable property.

In [ ]:

[FactType("Azure.Plan")]
public record Plan(AzureEnvironment environment, string planId);

[FactType("Azure.SubscriptionPlan")]
public record SubscriptionPlan(Subscription subscription, Plan plan, SubscriptionPlan[] prior);

Renderer.RenderTypes(typeof(Plan), typeof(SubscriptionPlan))

The user can activate and deactivate their subscription.
Azure can also choose to suspend and reinstate the subscription.
Finally, the user periodically renews their subscription.
Renewal occurs within a monthly period.

In [ ]:

[FactType("Azure.Subscription.Activate")]
public record Activate(Subscription subscription, DateTime activatedAt);

[FactType("Azure.Subscription.Deactivate")]
public record Deactivate(Activate activate, DateTime deactivatedAt);

[FactType("Azure.Subscription.Suspend")]
public record Suspend(Subscription subscription, DateTime suspendedAt);

[FactType("Azure.Subscription.Reinstate")]
public record Reinstate(Suspend suspend, DateTime reinstatedAt);

[FactType("Azure.Month")]
public record Month(AzureEnvironment environment, int year, int month);

[FactType("Azure.Subscription.Renew")]
public record Renew(Subscription subscription, Month month, DateTime renewedAt);

Renderer.RenderTypes(typeof(Suspend), typeof(Reinstate), typeof(Renew), typeof(Activate), typeof(Deactivate))